<a href="https://colab.research.google.com/github/juantomasprojects/codebase2gemini/blob/main/analyze_codebase_with_gemini_1_5_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyze a codebase with the Vertex AI Gemini 1.5 Pro


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcode%2Fanalyze_codebase_with_gemini_1_5_pro.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/code/analyze_codebase_with_gemini_1_5_pro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Eric Dong](https://github.com/gericdong), [Aakash Gouda](https://github.com/aksstar)|

## Overview

Gemini 1.5 Pro introduces a breakthrough long context window of up to 1 million tokens that can help seamlessly analyze, classify and summarize large amounts of content within a given prompt. With its long-context reasoning, Gemini 1.5 Pro can analyze an entire codebase for deeper insights.

In this tutorial, you learn how to analyze an entire codebase with Gemini 1.5 Pro and prompt the model to:

- **Analyze**: Summarize codebases effortlessly.
- **Guide**: Generate clear developer getting-started documentation.
- **Debug**: Uncover critical bugs and provide fixes.
- **Enhance**: Implement new features and improve reliability and security.


## Getting Started

### Install Vertex AI SDK for Python


In [5]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform \
                                        gitpython \
                                        magika

### Restart runtime (Colab only)

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [7]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "abadia-1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [31]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
)

## Cloning a codebase

You will use repo [Online Boutique](https://github.com/GoogleCloudPlatform/microservices-demo) as an example in this notebook. Online Boutique is a cloud-first microservices demo application. The application is a web-based e-commerce app where users can browse items, add them to the cart, and purchase them. This application consists of 11 microservices across multiple languages.

In [ ]:
# The GitHub repository URL
repo_url = "https://github.com/LaAbadIAdelCrimen/abadia-gym"  # @param {type:"string"}

# The location to clone the repo
repo_dir = "./repo"

#### Define helper functions for processing GitHub repository

In [ ]:
import os
import shutil
from pathlib import Path
import requests
import git
import magika

m = magika.Magika()


def clone_repo(repo_url, repo_dir):
    """Clone a GitHub repository."""

    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    os.makedirs(repo_dir)
    git.Repo.clone_from(repo_url, repo_dir)


def extract_code(repo_dir):
    """Create an index, extract content of code/text files."""

    code_index = []
    code_text = ""
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, repo_dir)
            code_index.append(relative_path)

            file_type = m.identify_path(Path(file_path))
            if file_type.output.group in ("text", "code"):
                try:
                    with open(file_path, "r") as f:
                        code_text += f"----- File: {relative_path} -----\n"
                        code_text += f.read()
                        code_text += "\n-------------------------\n"
                except Exception:
                    pass

    return code_index, code_text


def get_github_issue(owner: str, repo: str, issue_number: str) -> str:
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
    }  # Set headers for GitHub API

    # Construct API URL
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"

    try:
        response_git = requests.get(url, headers=headers)
        response_git.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as error:
        print(f"Error fetching issue: {error}")  # Handle potential errors

    issue_data = response_git.json()
    if issue_data:
        return issue_data["body"]
    return ""

#### Create an index and extract content of a codebase

Clone the repo and create an index and extract content of code/text files.

In [3]:
clone_repo(repo_url, repo_dir)

code_index, code_text = extract_code(repo_dir)

In [4]:
# prompt: write a local file with the content of the code_text variable

with open("codebase.txt", "w") as f:
  f.write(code_text)



## Analyzing the codebase with Gemini 1.5 Pro

With its long-context reasoning, Gemini 1.5 Pro can process the codebase and answer questions about the codebase.

#### Load the Gemini 1.5 Pro model

Learn more about the [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [8]:
MODEL_ID = "gemini-1.5-pro-001"  # @param {type:"string"}

model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "You are a coding expert.",
        "Your mission is to answer all code related questions with given context and instructions.",
    ],
)

#### Define a helper function to generate a prompt to a code related question

In [9]:
def get_code_prompt(question):
    """Generates a prompt to a code related question."""

    prompt = f"""
    Questions: {question}

    Context:
    - The entire codebase is provided below.
    - Here is an index of all of the files in the codebase:
      \n\n{code_index}\n\n.
    - Then each of the files is concatenated together. You will find all of the code you need:
      \n\n{code_text}\n\n

    Answer:
  """

    return prompt

### 1. Summarizing the codebase


Generate a summary of the codebase.

In [10]:
question = """
  Give me a summary of this codebase, and tell me the top 3 things that I can learn from it.
"""

prompt = get_code_prompt(question)
contents = [prompt]

# Generate text using non-streaming method
response = model.generate_content(contents)

# Print generated text and usage metadata
print(f"\nAnswer:\n{response.text}")
print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')
print(f"\nFinish reason:\n{response.candidates[0].finish_reason}")
print(f"\nSafety settings:\n{response.candidates[0].safety_ratings}")


Answer:
This codebase is an OpenAI gym environment for simulating the classic Spanish adventure game "La Abadía del Crimen" ("The Abbey of Crime"). It also includes several AI agents that interact with this environment, ranging from simple random agents to more sophisticated deep Q-learning agents.

Here's a summary of the codebase:

**1.  gym_abadia**: This is the core of the project, implementing the OpenAI gym environment for "The Abbey of Crime." It defines the game's state space (e.g., positions of characters, game variables), action space (possible actions the agent can take), reward function (how the agent is rewarded for its actions), and game logic. 

**2.  Agents**: The codebase includes several Python scripts implementing different agents for interacting with the "Abadia" environment. 
    * **agentv1.py**: Implements a "Do Nothing" agent, essentially a baseline that takes random actions.
    * **agentv2_qlearning.py & agentv3_qlearning.py**: Implement simple Q-learning age

### 2. Creating a developer getting started guide

Generate a getting started guide for developers. This sample uses the streaming option to generate the content.

In [ ]:
# prompt: write a local file with the content of the prompt variable

with open('prompt.txt', 'w') as f:
  f.write(prompt)


In [15]:
question = """
  Provide a getting started guide to onboard new developers to the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import gym
import gym_abadia

env = gym.make

('abadia-v0')
```

To onboard new developers to this

 codebase, follow these steps:

**1. Setup**

* **Clone the Repository:** `git clone https://github.com/LaAbad

IAdelCrimen/abadia-gym.git`
* **Install Python 3:** Ensure Python 3 is installed on your system.
*

...

 to the `README.md` file for instructions on running other agents.

**4. Code Structure**

* **`gym_abadia`:** Contains the Gym environment code.
* **`AbadIA`:** Houses different AI agent implementations (DQN, NGDQN, VDQN).


* **`tools`:**  Scripts for managing datasets, checkpoints, and training processes.
* **`k8s`:** Kubernetes configurations for running the project in a containerized environment.

**5. Contributing**

* **Issues:**  Report bugs or suggest improvements by opening issues on the GitHub repository

.
* **Pull Requests:** Contribute code changes through pull requests. Ensure your code follows the existing coding style and includes tests.

**6. Resources**

* **GitHub Repository:**  https://github.com/LaAbadIAdelCrimen/abadia-gym
* **OpenAI Gym Documentation

:**  https://gym.openai.com/docs/

**7. Additional Notes**

* The `README.md` file provides a starting point for understanding the project.
* Explore the code and experiment with different agents and settings.
* The `tools` directory contains scripts that can be helpful

 for data management and training.
* Use the Kubernetes configurations in the `k8s` directory for deploying the project.


### 3. Finding bugs

Find the top 3 most severe issues in the codebase.

In [14]:
question = """
  Find the top 3 most severe issues in the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

Let

's break down the top 3 issues within this codebase, focusing on

 maintainability, scalability, and best practices.

**1. Tight Coupling and

 Lack of Modular Design**

   * **Problem:** Many components (especially the agents and the environment) are intertwined. Functions within `AbadiaEnv2`

 directly manipulate agent-specific variables (e.g., `self.valMovs`, `self.wallMovs`, `self.perMovs`).

...

.).
   * **Solution:**
     * **Consistent Logging:** Standardize on the `logging` module. Set appropriate log levels (DEBUG, INFO, WARNING, ERROR, CRITICAL). This makes it easier to filter and manage logs based on their severity.
     * **Advanced Logging:** Explore more advanced logging configurations

 and tools (e.g., log rotation, centralized logging servers) to handle larger volumes of logs.

**Beyond the Top 3**

* **State Representation:** The way the game state is converted to a vector (`state2vector` function) could be improved for clarity and efficiency. Consider using data structures

 that more naturally align with the game's elements (perhaps a dictionary or a custom object). 
* **Multithreading:** The use of threads in `agentv6_ngdqn.py` for uploading models is a good start, but more systematic multithreading or multiprocessing could potentially be applied to speed up

 training or other compute-intensive parts of the code.
* **Code Organization:** You might benefit from a more structured project directory layout. Consider organizing code into logical modules and packages, separating core functionality from tests, scripts, and configuration.

By addressing these issues, you'll significantly enhance the codebase's

 quality, making it easier to understand, maintain, and extend in the future. 


### 4. Fixing bug

Find the most severe issue in the codebase that can be fixed and provide a code fix for it.


In [16]:
question = """
  Find the most severe bug in the codebase that you can provide a code fix for.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

diff
--- a/AbadIA/VDQN.py
+++ b

/AbadIA/VDQN.py
@@ -1579

,7 +1579,7 @@
             actionType = "E"
             self.env.calculated_predictions = []
             self

.env.final_predictions = []
-        else:
+        elif self.env.playing is True:
             predictions = self.model

.predict(vector.reshape(1,1,71)).reshape(9)
             logging.info(predictions)
             self.env.predictions = predictions
@@ -1612,6 +161

2,9 @@
         self.env.action_predictions = int(action)
         self.env.action_type = actionType
 
+        # in playing mode we only want to return a prediction
+

        if (self.env.playing is True):
+            action =  np.argmax(predictions)
         return action
 
     def remember(self, state, action, reward, new_state, done):

```

### 5. Implementing a feature request using Function Calling

Generate code to implement a feature request.

Get feature request text from GitHub Issue

In [17]:
# Function declaration with detailed docstring
extract_details_from_url_func = FunctionDeclaration(
    name="extract_details_from_url",
    description="Extracts owner, repository name, and issue number details from a GitHub issue URL",
    parameters={
        "type": "object",
        "properties": {
            "owner": {
                "type": "string",
                "description": "The owner of the GitHub repository.",
            },
            "repo": {
                "type": "string",
                "description": "The name of the GitHub repository.",
            },
            "issue_number": {
                "type": "string",
                "description": "The issue number to fetch the body of.",
            },
        },
    },
)

# Tool definition
extraction_tool = Tool(function_declarations=[extract_details_from_url_func])

FEATURE_REQUEST_URL = (
    "https://github.com/GoogleCloudPlatform/microservices-demo/issues/2205"
)

# Prompt content
prompt_content = f"What is the feature request of the following {FEATURE_REQUEST_URL}"

# Model generation with tool usage
response = model.generate_content(
    [prompt_content],
    generation_config=GenerationConfig(temperature=0),
    tools=[extraction_tool],
)
# Extract parameters from model response
function_call = response.candidates[0].function_calls[0]

# Fetch issue details from GitHub API if function call matches
if function_call.name == "extract_details_from_url":
    issue_body = get_github_issue(
        function_call.args["owner"],
        function_call.args["repo"],
        function_call.args["issue_number"],
    )

IPython.display.Markdown(f"Feature Request:\n{issue_body}")

Feature Request:
### Describe request or inquiry 
helm chart frontend-external support config service type nodeport, like this
```
helm install xxx --set frontend.service.type=NodePort
```

### What purpose/environment will this feature serve? 

This feature enables quick access to the front-end web interface of microservices without the need for additional configuration work.

I hope to quickly access the web interface after deploying this microservice in the local environment, without the need for additional loadbalancer or ingress configuration, just nodeport is enouth.

But now that the deployment is complete, I must manually edit the service and modify the nodeport. If Helm provides parameters to set the nodeport, I don't need to。


Use the GitHub Issue text to implement the feature request

In [19]:
# Combine feature request with URL and get code prompt
question = (
    "Implement the following feature request" + FEATURE_REQUEST_URL + "\n" + issue_body
)

prompt = get_code_prompt(question)

# Generate code response
response = model.generate_content([prompt])
IPython.display.Markdown(response.text)  # Display in Markdown format

```diff
--- a/kubernetes/abadia-v6/charts/abadia-agent-ng/templates/service.yaml
+++ b/kubernetes/abadia-agent-ng/templates/service.yaml
@@ -7,4 +7,4 @@
   ports:
   - protocol: TCP
     port: 80
-    targetPort: 4477
+    targetPort: {{ .Values.service.targetPort }}
   type: {{ .Values.service.type }}

```

### 6. Creating a troubleshooting guide

Create a troubleshooting guide to help resolve common issues.

In [20]:
question = """
    Provide a troubleshooting guide to help resolve common issues.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import random
import numpy as np
import logging
import json


import gzip
from math import hypot
from math import atan2
import pickle


import os
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import Adam


from collections import deque

# TODO JT:
# 1) Need a method to fill the memory with actions
# 2) Need a method to

...

 memory doesn't have enough data (at least 32 samples) for the `replay_game` function. 
* **Solution:**
    - Let the agent play more episodes and collect more data before attempting to train.
    - Consider increasing the `maxlen` parameter of the `deque` used

 for memory, if you want to store more actions.
    - If loading actions from files, ensure the files are correctly read and loaded into memory. 

**5. General Tips**

* **Logging:** Carefully review the log messages generated by the agents. These provide valuable information about the agent's actions

, rewards, and potential errors encountered.
* **Debugging:** Use a debugger to step through the code and understand its execution flow.
* **Code Structure:**  Familiarise yourself with the code structure and how different components (agent, environment, training/playing modes) interact.
* **Experimentation:** Try

 different hyperparameter values (gamma, epsilon, learning rate) and model architectures to see their impact on the agent's performance. 

This troubleshooting guide provides a starting point for resolving common issues. You may encounter other issues specific to your environment or use case. In such cases, carefully analyze error messages, log files

, and the code's execution flow to pinpoint the source of the problem and devise a solution. 


### 7. Making the app more reliable

Recommend best practices to make the application more reliable.


In [21]:
question = """
  How can I make this application more reliable? Consider best practices from https://www.r9y.dev/
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

The

 provided codebase is quite large and complex, making a comprehensive reliability analysis challenging without

 a specific area of focus. However, based on the files you shared and the

 link to best practices from R9y.dev, here's a general approach to improving the reliability of this application, focusing on common areas of improvement in

 machine learning applications:

**1. Input Validation and Handling**

* **Robust data handling:**
    * The code already includes some basic error handling for

...

 clearly defined responsibilities. 
    * Improve code readability, maintainability, and testability.

* **Unit tests:**
    * Write unit tests to cover the core functions and logic of the codebase, especially for critical components like state vectorization, action validation, and model training/saving. 


    * Ensure that the code functions as expected and helps detect regressions during development.

**5. Cloud Integration (if applicable)**

* **Error handling for cloud operations:**
    * The code uses Google Cloud Storage. 
    * Ensure robust error handling for cloud operations like downloading/uploading files, handling

 potential network issues, and authentication failures.
    * Implement retries and exponential backoff for transient errors.

* **Resource management:**
    * Implement efficient resource management for cloud resources. 
    * Release resources promptly when no longer needed to avoid unnecessary costs.

**Specific Recommendations based on R9y.

dev**

* **Defensive programming:** Apply defensive programming principles throughout the codebase, checking for potential error conditions and handling them gracefully.
* **Use assertions:** Use assertions liberally to verify assumptions about the code and data.
* **Limit dependencies:** Minimize the number of external libraries and dependencies to reduce the potential attack

 surface and complexity.

**Note:** These suggestions are general guidelines. Prioritizing and implementing them should be based on a thorough understanding of the application's specific requirements and the criticality of its reliability.


### 8. Making the app more secure

Recommend best practices to make the application more secure.

In [22]:
question = """
  How can you secure the application?
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

This

 codebase represents a reinforcement learning project for the game "The Abbey of Crime"

 (AbadIA), leveraging OpenAI Gym and Google Cloud Platform (GCP

). While the code focuses on the agent's learning and interaction with the game, security considerations seem to be largely absent. Here's a breakdown of potential

 vulnerabilities and how to address them:

**1. Communication with the Game Engine**

* **Unencrypted Communication:** The code uses HTTP requests to communicate with

...

Solution:** Use tools like `pip-audit` or `safety` to scan for vulnerable dependencies. Keep your environment up to date using `pip install -r requirements.txt --upgrade`. 

* **Logging:** The code uses basic logging, but it's important to consider what data is being logged and whether

 any of it is sensitive.
    * **Solution:** Avoid logging sensitive information like passwords or API keys. Configure logging levels appropriately (INFO, DEBUG, etc.) and be mindful of the storage location and retention policies for logs.

**4. Kubernetes Security**

* **Container Images:** The Kubernetes YAML files (`

k8s/` directory) specify container images. These images could have vulnerabilities if not properly secured.
    * **Solution:** Use trusted base images. Scan images for vulnerabilities using tools like Clair, Anchore, or Trivy. Implement image signing to verify the integrity of images.

* **Pod Security:**

 Kubernetes pods should be configured with appropriate security contexts.
    * **Solution:** Define resource limits to prevent resource exhaustion. Use security contexts to control privileges, network access, and filesystem access within the pod. 

**Implementation Notes:**

* Secure coding practices should be integrated throughout the codebase, not just in isolated

 modules.
* Regular security testing and vulnerability scanning are essential to identify and mitigate risks.
* Consider using security tools and libraries designed for Python and GCP.

By addressing these vulnerabilities, you can significantly enhance the security of this reinforcement learning application and protect its data, models, and infrastructure. 


### 9. Learning the codebase

Create a quiz about the concepts used in the codebase.

In [23]:
question = """
  Create a quiz about the concepts used in my codebase to help me solidify my understanding.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import json
import random

# Questions and answers are stored as a

 list of dictionaries
quiz_data = [
    {
        "question

": "What type of learning algorithm is primarily used in 'agentv2_qlearning.py' and 'agentv3_qlearning.py'

?",
        "options": ["a) Deep Q-learning", "b) Q-learning", "c) Monte Carlo Tree Search", "d)

...

QN.py'"],
        "answer": "b"
    }
]

# Function to present a question and get the user's answer
def ask_question(question_data):
    print(question_data["question"])
    for option in question_data["options"]:


        print(option)
    while True:
        user_answer = input("Enter your answer (a, b, c, or d): ").lower()
        if user_answer in ["a", "b", "c", "d"]:
            break
        print("Invalid input. Please

 enter a, b, c, or d.")
    return user_answer

# Main quiz loop
def run_quiz(quiz_data):
    score = 0
    random.shuffle(quiz_data)  # Shuffle the questions
    for question_data in quiz_data:


        user_answer = ask_question(question_data)
        if user_answer == question_data["answer"]:
            print("Correct!\n")
            score += 1
        else:
            print(f"Incorrect. The answer was {question_data['options'][ord(

question_data['answer']) - ord('a')]}.\n")
    print(f"You got {score} out of {len(quiz_data)} questions right.")

# Run the quiz
if __name__ == "__main__":
    run_quiz(quiz_data)
``` 

### 10. Creating a quickstart tutorial

Create an end-to-end quickstart tutorial for a specific component.


In [24]:
question = """
  Please write an end-to-end quickstart tutorial that introduces AlloyDB,
  shows how to configure it with the CartService,
  and highlights key capabilities of AlloyDB in context of the Online Boutique application.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

```

python
import gym
import gym_abadia
import numpy as np


import os
import argparse
import random
import json
import AbadIA.

NGDQN

from google.cloud import sql_v1beta4
from google.cloud import storage

# Initialize Google Cloud Storage client
storage_

client = storage.Client()

def init_env(env):
    """
    Initializes the environment with arguments passed from the command line.


...

Update the Online Boutique CartService:** Modify your existing `CartService` class (or create a new one) to use AlloyDB for persistence. Implement the methods you need for cart operations using the `google.cloud.sql_v1beta4` library to connect to AlloyDB and execute SQL queries.

3

. **Modify the code:** Replace the placeholder `CartService` class and its methods in the provided example code with your actual implementation.

4. **Install the necessary libraries:**
   ```bash
   pip install google-cloud-sql google-cloud-storage
   ```

5. **Run the example

:**
   ```bash
   python3 your_script.py \
     --server your-alloydb-connection-name \
     --database your-alloydb-database \
     --user your-alloydb-user \
     --password your-alloydb-password \
     --

bucket your-gcs-bucket-name 
   ```

**Important Notes:**

- This is a simplified example. You'll need to adapt it to your specific Online Boutique implementation.
- Remember to follow security best practices for managing your AlloyDB credentials (using secrets management).
- Explore other features of

 AlloyDB, like its integration with Cloud Spanner, for more advanced use cases. 


### 11. Creating a Git Changelog Generator

Understanding changes made between Git commits and highlighting the most important aspects of the changes.

In [25]:
### Fetches commit IDs from a local Git repository on a specified branch.

repo = git.Repo(repo_dir)
branch_name = "main"
commit_ids = [
    commit.hexsha for commit in repo.iter_commits(branch_name)
]  # A list of commit IDs (SHA-1 hashes) in reverse chronological order (newest first)

if len(commit_ids) >= 2:
    diff_text = repo.git.diff(commit_ids[0], commit_ids[1])

    question = """
      Given the above git diff output, Summarize the important changes made.
    """

    prompt = diff_text + question + code_text
    contents = [prompt]

    responses = model.generate_content(contents, stream=True)
    for response in responses:
        IPython.display.Markdown(response.text)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-list main --
  stderr: 'fatal: bad revision 'main'
'

## Conclusion

In this tutorial, you've learned how to use the Gemini 1.5 Pro to analyze a codebase and prompt the model to:

- Summarize codebases effortlessly.
- Generate clear developer getting-started documentation.
- Uncover critical bugs and provide fixes.
- Implement new features and improve reliability and security.
- Understanding changes made between Git commits